In [1]:
import cv2 as cv
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#directories
dataset_dir = 'dataset'
output_dir ="output"
g_matrix_filename = "G.npy"

In [3]:
#store the computed images on disk
save_output = False

In [4]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [5]:
imgs = load_images_from_folder(dataset_dir)
n = len(imgs)

In [6]:
G = np.load(os.path.join(output_dir,g_matrix_filename))

In [7]:
i=0
j=0
print(G[3*i:3*(i+1),3*j:3*(j+1)])

[[-5. -0. -0.]
 [-0. -5. -0.]
 [-0. -0. -5.]]


In [10]:
 u,d,v = np.linalg.svd(G)

In [11]:
print(G.shape, u.shape, d.shape, v.shape)

(60, 60) (60, 60) (60,) (60, 60)


In [26]:
#u: left eigenvectors
#d: eigenvalues
#v: right eigenvectors

In [13]:
print(d)

[6.15494760e+03 6.13095010e+03 5.37487912e+03 5.36914858e+03
 3.93733344e+03 3.92289165e+03 3.23191892e+03 3.21126173e+03
 2.71075425e+03 2.69097109e+03 2.22803695e+03 2.22137164e+03
 1.86155656e+03 1.84687425e+03 1.08724633e+03 1.07629866e+03
 3.23573052e+02 2.00941120e+02 1.94959963e+02 1.57336726e+02
 6.79801193e+00 6.21669676e+00 5.99911911e+00 5.78348084e+00
 5.40029126e+00 5.11445610e+00 5.04217889e+00 4.68417800e+00
 4.55776362e+00 4.46039411e+00 4.37586450e+00 4.26111692e+00
 3.73564625e+00 3.51085560e+00 3.43619965e+00 2.60147596e+00
 2.30296681e+00 1.86142950e+00 1.24831989e+00 6.12601917e-01
 9.17975917e-02 7.73966647e-02 2.66582424e-02 2.48502214e-02
 1.28056651e-02 1.09492984e-02 1.07160260e-02 8.73528418e-03
 8.07777362e-03 7.40839476e-03 4.31329551e-03 3.22234155e-03
 1.79630107e-03 1.16222510e-03 1.09681653e-03 2.22970115e-04
 1.17239157e-04 5.50231675e-06 3.97314804e-06 5.31541490e-07]


In [15]:
 np.argsort(d)

array([59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43,
       42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26,
       25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10,  9,
        8,  7,  6,  5,  4,  3,  2,  1,  0], dtype=int64)

In [27]:
# we need to choose the 3 smallest eigenvalues

In [16]:
U = u[:,-3:]

In [19]:
u

array([[-3.99910136e-02,  5.19761672e-01, -5.93248895e-02, ...,
         1.99141807e-05, -7.36100259e-05,  2.95066217e-05],
       [-5.03690002e-04,  3.15857784e-02, -4.04692724e-03, ...,
        -8.50428726e-05,  5.86778101e-05,  2.39881706e-04],
       [ 2.70897445e-04,  8.90252280e-05, -7.31833614e-06, ...,
        -2.24820077e-01, -4.95233101e-02, -2.69350029e-01],
       ...,
       [ 2.08028260e-02,  5.15701227e-01, -5.95958543e-02, ...,
         2.47752852e-05, -7.21341143e-05,  2.87290642e-05],
       [ 7.85527329e-04,  4.12941636e-02, -5.20846492e-03, ...,
        -8.47649586e-05,  5.83799290e-05,  2.39688844e-04],
       [ 2.65566613e-04,  3.90669329e-05, -6.99280981e-07, ...,
        -2.25530417e-01, -6.93268782e-02, -2.63651114e-01]])

In [20]:
U

array([[ 1.99141807e-05, -7.36100259e-05,  2.95066217e-05],
       [-8.50428726e-05,  5.86778101e-05,  2.39881706e-04],
       [-2.24820077e-01, -4.95233101e-02, -2.69350029e-01],
       [ 6.34534709e-06, -7.56936767e-05,  2.68634702e-05],
       [-8.46774723e-05,  5.94193468e-05,  2.41015164e-04],
       [-2.14716405e-01,  1.61591183e-02, -2.99526075e-01],
       [-1.52449703e-05, -7.38689612e-05,  2.24099389e-05],
       [-8.61961062e-05,  5.94055008e-05,  2.41627111e-04],
       [-1.76076151e-01,  1.03170765e-01, -3.34153293e-01],
       [-3.75693932e-05, -6.16084599e-05,  1.17071286e-05],
       [-8.56087534e-05,  6.01315778e-05,  2.41651816e-04],
       [-9.68471150e-02,  1.93101812e-01, -3.51862865e-01],
       [-5.77266877e-05, -3.32807765e-05, -3.78942609e-06],
       [-8.42830263e-05,  5.99626549e-05,  2.39931135e-04],
       [ 4.47013127e-02,  2.59221066e-01, -3.38815127e-01],
       [-6.24456016e-05,  3.55200149e-06, -1.88208275e-05],
       [-8.37485100e-05,  6.18653093e-05

In [21]:
res = [ U[i*3:(i+1)*3,:] for i in range(n)]

In [24]:
len(res)

20

In [25]:
res[0].shape

(3, 3)